# Aggregate Data into Weekly Averages


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


def process_weekly_averages(data_path, output_path, start_date, end_date):
    """
    Process stock data into weekly averages based on 7-day increments using Close price.

    Parameters:
    - data_path: Path to the input CSV file
    - output_path: Path for the output CSV file
    - start_date: Start date string (format: 'YYYY-MM-DD')
    - end_date: End date string (format: 'YYYY-MM-DD')
    """
    # Read the CSV file
    df = pd.read_csv(data_path)

    # Ensure the first column is treated as Date, regardless of its header
    cols = list(df.columns)
    if len(cols) == 0:
        raise ValueError("Input file has no columns")
    cols[0] = 'Date'
    df.columns = cols

    # Find a Close-like column
    def find_close_column(columns):
        def norm(c):
            return c.strip().lower()
        # 1) exact 'close'
        for c in columns:
            if norm(c) == 'close':
                return c
        # 2) common variants
        variants = {
            'adj close', 'adj_close', 'closing price', 'close price', 'price close'
        }
        for c in columns:
            if norm(c) in variants:
                return c
        # 3) contains 'close'
        for c in columns:
            if 'close' in norm(c):
                return c
        return None

    close_col = find_close_column(df.columns)
    if close_col is None:
        raise ValueError("Couldn't find a 'Close' column in the file. Ensure a column named 'Close' exists.")

    # Keep only Date and Close, normalize column name
    df = df[['Date', close_col]].rename(columns={close_col: 'Close'})

    # Parse dates and Close values
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    df = df.dropna(subset=['Date', 'Close'])

    # Filter data between start and end dates
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    df = df[(df['Date'] >= start) & (df['Date'] <= end)]

    # Sort by date
    df = df.sort_values('Date')

    # Create week bins based on 7-day increments from start date
    df['Days_From_Start'] = (df['Date'] - start).dt.days
    df['Week_Number'] = df['Days_From_Start'] // 7

    # Calculate the week start/end dates for each entry
    df['Week_Start'] = start + pd.to_timedelta(df['Week_Number'] * 7, unit='D')
    df['Week_End'] = df['Week_Start'] + pd.Timedelta(days=6)

    # Group by week and calculate average close price
    weekly_avg = (
        df.groupby(['Week_Number', 'Week_Start', 'Week_End'])
          .agg({'Close': 'mean', 'Date': 'count'})
          .reset_index()
    )

    # Rename columns
    weekly_avg.columns = ['Week_Number', 'Week_Start', 'Week_End', 'Avg_Close_Price', 'Trading_Days']

    # Format dates
    weekly_avg['Week_Start'] = weekly_avg['Week_Start'].dt.strftime('%Y-%m-%d')
    weekly_avg['Week_End'] = weekly_avg['Week_End'].dt.strftime('%Y-%m-%d')

    # Save to CSV
    weekly_avg.to_csv(output_path, index=False)

    print(f"Processed {len(df)} daily records into {len(weekly_avg)} weekly averages")
    print(f"Output saved to: {output_path}")

    return weekly_avg

# Aggregate SPX


In [29]:
# Process SPX data from Monday Sept 28, 1998 to Sunday Sept 30, 2018
result_spx = process_weekly_averages(
    data_path='data/SPX.csv',
    output_path='data/SPX_weekly_averages.csv',
    start_date='1998-09-28',
    end_date='2018-09-30'
)

# Display examples
print("\nSPX — First 10 weeks:")
print(result_spx.head(10))
print("\nSPX — Last 10 weeks:")
print(result_spx.tail(10))

Processed 5035 daily records into 1044 weekly averages
Output saved to: data/SPX_weekly_averages.csv

SPX — First 10 weeks:
   Week_Number  Week_Start    Week_End  Avg_Close_Price  Trading_Days
0            0  1998-09-28  1998-10-04      1020.741992             5
1            1  1998-10-05  1998-10-11       977.532007             5
2            2  1998-10-12  1998-10-18      1020.390015             5
3            3  1998-10-19  1998-10-25      1069.078027             5
4            4  1998-10-26  1998-11-01      1078.069995             5
5            5  1998-11-02  1998-11-08      1123.193994             5
6            6  1998-11-09  1998-11-15      1124.567969             5
7            7  1998-11-16  1998-11-22      1147.165991             5
8            8  1998-11-23  1998-11-29      1187.599976             4
9            9  1998-11-30  1998-12-06      1167.408008             5

SPX — Last 10 weeks:
      Week_Number  Week_Start    Week_End  Avg_Close_Price  Trading_Days
1034       

# Aggregate KOSPI


In [6]:
# Process KOSPI data from Monday Nov 3, 1997 to Wednesday Nov 7, 2007
result = process_weekly_averages(
    data_path='data/KOSPI.csv',
    output_path='data/KOSPI_weekly_averages.csv',
    start_date='1987-11-03',
    end_date='2007-11-07'
)

# Display first few rows
print("\nFirst 10 weeks:")
print(result.head(10))
print("\nLast 10 weeks:")
print(result.tail(10))

Processed 4927 daily records into 1044 weekly averages
Output saved to: data/KOSPI_weekly_averages.csv

First 10 weeks:
   Week_Number  Week_Start    Week_End  Avg_Close_Price  Trading_Days
0            0  1987-11-03  1987-11-09       498.096000             5
1            1  1987-11-10  1987-11-16       473.162000             5
2            2  1987-11-17  1987-11-23       472.278000             5
3            3  1987-11-24  1987-11-30       463.972000             5
4            4  1987-12-01  1987-12-07       486.998000             5
5            5  1987-12-08  1987-12-14       471.578000             5
6            6  1987-12-15  1987-12-21       485.574000             5
7            7  1987-12-22  1987-12-28       509.923333             3
8            8  1987-12-29  1988-01-04       532.040000             1
9            9  1988-01-05  1988-01-11       541.380000             5

Last 10 weeks:
      Week_Number  Week_Start    Week_End  Avg_Close_Price  Trading_Days
1034         1035  20

# Aggregate NIKKEI225


In [8]:
# Process NIKKEI225 data from Monday Nov 3, 1979 to Wednesday Nov 7, 1999
result = process_weekly_averages(
    data_path='data/NIKKEI225.csv',
    output_path='data/NIKKEI225_weekly_averages.csv',
    start_date='1979-12-24',
    end_date='1999-12-31'
)

# Display first few rows
print("\nFirst 10 weeks:")
print(result.head(10))
print("\nLast 10 weeks:")
print(result.tail(10))

Processed 5018 daily records into 1044 weekly averages
Output saved to: data/NIKKEI225_weekly_averages.csv

First 10 weeks:
   Week_Number  Week_Start    Week_End  Avg_Close_Price  Trading_Days
0            0  1979-12-24  1979-12-30        6539.7680             5
1            1  1979-12-31  1980-01-06        6560.1600             1
2            2  1980-01-07  1980-01-13        6583.4560             5
3            3  1980-01-14  1980-01-20        6654.6025             4
4            4  1980-01-21  1980-01-27        6711.9380             5
5            5  1980-01-28  1980-02-03        6771.3680             5
6            6  1980-02-04  1980-02-10        6794.2620             5
7            7  1980-02-11  1980-02-17        6832.9025             4
8            8  1980-02-18  1980-02-24        6777.0860             5
9            9  1980-02-25  1980-03-02        6755.3880             5

Last 10 weeks:
      Week_Number  Week_Start    Week_End  Avg_Close_Price  Trading_Days
1034         1035

# CPI for Inflation

In [9]:
%pip install cpi

   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
   -- ------------------------------------- 1.0/18.1 MB 8.1 MB/s eta 0:00:03
   ------- -------------------------------- 3.4/18.1 MB 9.6 MB/s eta 0:00:02
   ------------- -------------------------- 6.0/18.1 MB 10.5 MB/s eta 0:00:02
   -------------------- ------------------- 9.2/18.1 MB 11.6 MB/s eta 0:00:01
   -------------------------- ------------- 12.1/18.1 MB 12.0 MB/s eta 0:00:01
   --------------------------------- ------ 14.9/18.1 MB 12.5 MB/s eta 0:00:01
   ---------------------------------------  17.8/18.1 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 18.1/18.1 MB 12.3 MB/s  0:00:01
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Calculate Adjusted for Inflation


In [8]:
import cpi

def adjust_to_cpi_2025(csv_path, current_year, output_path=None):
    """
    Adjust Avg_Close_Price to current year dollars using CPI index.
    Uses the year from Week_End as the baseline year for each row.
    Appends 'Avg_Close_Price_current_year' column and saves the updated CSV.
    
    Parameters:
    - csv_path: Path to the input CSV file with Avg_Close_Price column
    - output_path: Path for the output CSV file (defaults to overwriting input)
    """
    if output_path is None:
        output_path = csv_path
    
    # Read the CSV
    df = pd.read_csv(csv_path)
    
    if 'Avg_Close_Price' not in df.columns:
        raise ValueError("CSV must contain 'Avg_Close_Price' column")
    if 'Week_End' not in df.columns:
        raise ValueError("CSV must contain 'Week_End' column")
    
    # Extract year from Week_End
    df['Year'] = pd.to_datetime(df['Week_End']).dt.year
    
    # Adjust each price to 2025 dollars
    adjusted_prices = []
    for idx, row in df.iterrows():
        original_price = row['Avg_Close_Price']
        year = row['Year']
        
        try:
            # Convert from year-dollars to 2025-dollars
            adjusted = cpi.inflate(original_price, year, to=current_year)
            adjusted_prices.append(adjusted)
        except Exception as e:
            print(f"Warning: Could not adjust price for year {year}: {e}")
            adjusted_prices.append(None)
    
    df[f'Avg_Close_Price_{current_year}'] = adjusted_prices
    
    # Remove temporary Year column
    df = df.drop(columns=['Year'])
    
    # Save the updated CSV
    df.to_csv(output_path, index=False)
    
    print(f"CPI-adjusted data saved to: {output_path}")
    print(f"New column 'Avg_Close_Price_2025' appended with {len([p for p in adjusted_prices if p is not None])} valid adjustments")
    
    return df

# Set Year


In [9]:
current_year = 2024

# Adjust for Inflation SPX

In [ ]:
# Apply CPI adjustment to SPX weekly averages
print(f"Adjusting SPX weekly averages to {current_year} dollars...")
output_path = 'data/SPX_weekly_averages.csv'
spx_adjusted = adjust_to_cpi_2025(output_path, current_year)
spx_adjusted.to_csv(output_path, index=False)
# Display sample with original and adjusted prices
print("\nSPX sample (first 5 rows):")
print(spx_adjusted[['Week_Start', 'Week_End', 'Avg_Close_Price', f'Avg_Close_Price_{current_year}']].head())

Adjusting SPX weekly averages to 2024 dollars...
CPI-adjusted data saved to: data/SPX_weekly_averages.csv
New column 'Avg_Close_Price_2025' appended with 1044 valid adjustments

SPX sample (first 5 rows):
   Week_Start    Week_End  Avg_Close_Price  Avg_Close_Price_2025
0  1998-09-28  1998-10-04      1020.741992           1964.389784
1  1998-10-05  1998-10-11       977.532007           1881.233360
2  1998-10-12  1998-10-18      1020.390015           1963.712413
3  1998-10-19  1998-10-25      1069.078027           2057.411149
4  1998-10-26  1998-11-01      1078.069995           2074.715943


# Adjust for Inflation KOSPI

In [ ]:
# Apply CPI adjustment to KOSPI weekly averages
print(f"Adjusting KOSPI weekly averages to {current_year} dollars...")
output_path = 'data/KOSPI_weekly_averages.csv'
kospi_adjusted = adjust_to_cpi_2025(output_path, current_year)
kospi_adjusted.to_csv(output_path, index=False)
# Display sample with original and adjusted prices
print("\nKOSPI sample (first 5 rows):")
print(kospi_adjusted[['Week_Start', 'Week_End', 'Avg_Close_Price', f'Avg_Close_Price_{current_year}']].head())

Adjusting KOSPI weekly averages to 2024 dollars...


In [22]:
len(spx_adjusted)

1044

In [2]:
def find_missing_weeks(csv_path, start_date, end_date):
    """
    Given a weekly CSV and a date range, list missing 7-day bins based on Week_Number.
    Assumes columns: Week_Number, Week_Start, Week_End.
    """
    df = pd.read_csv(csv_path)
    required = ['Week_Number', 'Week_Start', 'Week_End']
    for col in required:
        if col not in df.columns:
            raise ValueError(f"Missing required column '{col}' in {csv_path}")
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    total_weeks = int(np.floor((end - start).days / 7)) + 1
    expected = set(range(total_weeks))
    present = set(df['Week_Number'].dropna().astype(int))
    missing_numbers = sorted(expected - present)
    if missing_numbers:
        week_starts = [start + pd.Timedelta(days=7*n) for n in missing_numbers]
        week_ends = [ws + pd.Timedelta(days=6) for ws in week_starts]
        missing_df = pd.DataFrame({
            'Week_Number': missing_numbers,
            'Week_Start': [ws.strftime('%Y-%m-%d') for ws in week_starts],
            'Week_End': [we.strftime('%Y-%m-%d') for we in week_ends],
        })
    else:
        missing_df = pd.DataFrame(columns=['Week_Number', 'Week_Start', 'Week_End'])
    print(f"Missing weeks: {len(missing_df)} out of {total_weeks} total bins")
    if len(missing_df):
        print(missing_df.head(10))
    return missing_df

In [3]:

# Example: check SPX weekly CSV
missing_df = find_missing_weeks('data/KOSPI_weekly_averages.csv', '1987-11-03', '2007-11-07')

NameError: name 'pd' is not defined